# Getting Started With SQL and BigQuery

## Lecture

```python
!pip install google-cloud-bigquery
```

In [17]:
from google.cloud import bigquery

The first step in the workflow is to create a [Client](https://google-cloud.readthedocs.io/en/latest/bigquery/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client) object. As you'll soon see, this Client object will play a central role in retrieving information from BigQuery datasets.

In [18]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "bigquery_key.json"

In [19]:
# create a client object
client = bigquery.Client()

We'll work with a dataset of posts on [Hacker News](https://news.ycombinator.com/), a website focusing on computer science and cybersecurity news.

In BigQuery, each dataset is contained in a corresponding project. In this case, our hacker_news dataset is contained in the bigquery-public-data project. To access the dataset,

We begin by constructing a reference to the dataset with the dataset() method.
Next, we use the get_dataset() method, along with the reference we just constructed, to fetch the dataset.

In [20]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

We use the `list_tables()` method to list the tables in the dataset.

In [21]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:
    print(table.table_id)

full


Similar to how we fetched a dataset, we can fetch a table. In the code cell below, we fetch the full table in the hacker_news dataset.

In [22]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table('full')

# API request - fetch the table
table = client.get_table(table_ref)

In the next section, you'll explore the contents of this table in more detail. For now, take the time to use the image below to consolidate what you've learned so far. 

Remark: currently only have __full__ table

![image](https://storage.googleapis.com/kaggle-media/learn/images/biYqbUB.png)]


### Table schema

The structure of a table is called its schema. We need to understand a table's schema to effectively pull out the data we want.

In this example, we'll investigate the full table that we fetched above.

In [23]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'type of details (comment comment_ranking poll story job pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', N

Each SchemaField tells us about a specific column (which we also refer to as a __field__). In order, the information is:

* The __name__ of the column
* The __field type__ (or datatype) in the column
* The __mode__ of the column ('NULLABLE' means that a column allows NULL values, and is the default)
* A __description__ of the data in that column

The first field has the SchemaField:

`SchemaField('by', 'string', 'NULLABLE', "The username of the item's author.",())`

This tells us:

* the field (or column) is called __by__,
* the data in this field is __strings__,
* NULL values are allowed, and
* it contains the usernames corresponding to each item's author.

We can use the `list_rows()` method to check just the first five lines of of the full table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery <u>RowIterator</u> object that can quickly be converted to a pandas DataFrame with the `to_dataframe()` method.

```python
!pip install db-dtypes
```

In [24]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"If the crocodile looked him up on Google, we b...",<NA>,raxxorrax,<NA>,1633421535,2021-10-05 08:12:15+00:00,comment,28756662,28750122,<NA>,<NA>,<NA>
1,None,None,What exactly are you looking for? I think Pyto...,<NA>,abiro,<NA>,1569141387,2019-09-22 08:36:27+00:00,comment,21040311,21040141,<NA>,<NA>,<NA>
2,None,None,"Ironically, this very project might help out w...",<NA>,mjevans,<NA>,1505769703,2017-09-18 21:21:43+00:00,comment,15279716,15276626,<NA>,<NA>,<NA>
3,None,None,As you start to gain some experience it can be...,<NA>,every_other,<NA>,1538575027,2018-10-03 13:57:07+00:00,comment,18130207,18128477,<NA>,<NA>,<NA>
4,None,None,"That’s what I was referring to, yes. I heard o...",<NA>,manmal,<NA>,1615664155,2021-03-13 19:35:55+00:00,comment,26449260,26449237,<NA>,<NA>,<NA>


The `list_rows()` method will also let us look at just the information in a specific column. If we want to see the first five entries in the by column, for example, we can do that!

In [25]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


# Select, From & Where

## Lecture

### SELECT ... FROM

The most basic SQL query selects a single column from a single table. To do this,

* specify the column you want after the word __SELECT__, and then
* specify the table after the word __FROM__.

For instance, to select the Name column (from the pets table in the pet_records database in the bigquery-public-data project), our query would appear as follows:

![TABLE](https://storage.googleapis.com/kaggle-media/learn/images/c3GxYRt.png)

Note that when writing an SQL query, the argument we pass to __FROM__ is not in single or double quotation marks (' or "). It is in backticks (`).

### WHERE ...

BigQuery datasets are large, so you'll usually want to return only the rows meeting specific conditions. You can do this using the __WHERE__ clause.

The query below returns the entries from the Name column that are in rows where the Animal column has the text 'Cat'.

![IMAGE 2](https://storage.googleapis.com/kaggle-media/learn/images/HJOT8Kb.png)

### Example: What are all the U.S. cities in the OpenAQ dataset?

Now that you've got the basics down, let's work through an example with a real dataset. We'll use an OpenAQ dataset about air quality.

First, we'll set up everything we need to run queries and take a quick peek at what tables are in our database. (Since you learned how to do this in the previous tutorial, we have hidden the code. But if you'd like to take a peek, you need only click on the "Code" button below.)

In [26]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Build Google Cloud credentials
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "bigquery_key.json"

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project='bigquery-public-data')

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "openaq" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:
    print(table.table_id)

global_air_quality


The dataset contains only one table, called __global_air_quality__. We'll fetch the table and take a peek at the first few rows to see what sort of data it contains. 

In [27]:
# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table('global_air_quality')

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


Everything looks good! So, let's put together a query. Say we want to select all the values from the city column that are in rows where the country column is 'US' (for "United States").

In [28]:
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
        SELECT city FROM 'bigquery-public-data.openaq.global_air_quality'
        WHERE country = 'US'
        """

### Submitting the query to the dataset

We're ready to use this query to get information from the OpenAQ dataset. As in the previous tutorial, the first step is to create a Client object.

In [29]:
# Create a "Client" object
# client = bigquery.Client()

We begin by setting up the query with the `query()` method. We run the method with the default parameters, but this method also allows us to specify more complicated settings that you can read about in the documentation. We'll revisit this later.

In [30]:
# Set up the query
query_job = client.query(query)

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/brilliant-rhino-311215/jobs?prettyPrint=false: Access Denied: Project brilliant-rhino-311215: User does not have bigquery.jobs.create permission in project brilliant-rhino-311215.

Location: None
Job ID: 799c715c-4ea8-4bb4-944c-b9ded2ed2fff
